# Task 4: Classification of tweets self-reporting potential COVID19 cases

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 80 kB 7.8 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 596 kB 46.4 MB/s 
     |████████████████████████████████| 6.6 MB 34.3 MB/s 
     |████████████████████████████████| 287 kB 47.3 MB/s 
     |████████████████████████████████| 106 kB 50.1 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from transformers import RobertaTokenizerFast, RobertaModel
from transformers import AutoTokenizer, AutoModel
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "train.tsv"
val_filename = "valid.tsv"

In [4]:
# Load data
train = pd.read_csv(train_filename, sep="\t", names=["tweet_id", "user_id", "tweet", "label"])
validation = pd.read_csv(val_filename, sep="\t")

In [5]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (6465, 4) and validation data is (716, 4)


In [6]:
# Train top 5 rows
train.head().style.set_caption("Task 1: Train dataset")

,tweet_id,user_id,tweet,label
0,1239172732690014208,2391447188,We’re parking at the airport and my mom rolled down the window to speak to an attendant and my dad immediately said “we have the coronavirus sir”,0
1,1223737201030246402,1200539436167159809,I really didn’t expect this will go wide this way. I hope safety & health for all people of #Chine & whole world. We are just trying to show some support & respect to them as much we can especially doctors who bravely facing the dirty #coronaVirus.,0
2,1239385333319389185,838382730,"For those who believe they are immortal and continue to go out to the park without paying attention to the order to remain at home, these are the x-rays of a 28-year-old boy intubated in the ICU in my hospital for #coronavirus. Hint: the lungs are black, white is pneumonia",1
3,1236209435241938945,780855138,My flight from Jordan back to the US stops in Paris 😂 will I be quarantined? Stay tuned to find out 😂😂 #coronavirus,0
4,1233855551605440514,337103373,I went to the movies and the air was on. Now I'm out to eat and Olive Garden has the air on. I see these establishments are doing their best to fight the coronavirus.,0


## 2. Prepare the data - Clean & Prepare for Model

In [7]:
# Drop unwanted columns
train.drop(['tweet_id', 'user_id'], axis=1, inplace=True)
validation.drop(['tweet_id', 'user_id'], axis=1, inplace=True)

In [8]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [9]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [11]:
# Train top 5 rows after pre-processing
train[['label', 'clean_tweets']].head()

,label,clean_tweets
0,0,we ’ re parking at the airport and my mom rolled down the window to speak to an attendant and my dad immediately said “ we have the coronavirus sir ”
1,0,i really didn ’ t expect this will go wide this way . i hope safety & health for all people of <hashtag> chine </hashtag> & whole world . we are just trying to show some support & respect to them as much we can especially doctors who bravely facing the dirty <hashtag> corona virus </hashtag> .
2,1,"for those who believe they are immortal and continue to go out to the park without paying attention to the order to remain at home , these are the x - rays of a <number> - year - old boy intubated in the <allcaps> icu </allcaps> in my hospital for <hashtag> coronavirus </hashtag> . hint : the lungs are black , white is pneumonia"
3,0,my flight from jordan back to the us stops in paris 😂 will i be quarantined ? stay tuned to find out 😂 😂 <hashtag> coronavirus </hashtag>
4,0,i went to the movies and the air was on . now i am out to eat and olive garden has the air on . i see these establishments are doing their best to fight the coronavirus .


In [12]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [13]:
# Define BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

In [14]:
# Tokenize train and validation data
train_enc_sci = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc_sci = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [15]:
train_enc_sci.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [16]:
train_enc_sci.input_ids.shape, train_enc_sci.token_type_ids.shape, train_enc_sci.attention_mask.shape

(torch.Size([6465, 128]), torch.Size([6465, 128]), torch.Size([6465, 128]))

In [17]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [18]:
def get_dataloader(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.token_type_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [20]:
train_dataloader_sci = get_dataloader(train_enc_sci, torch.tensor(train['label'].to_list()))
valid_dataloader_sci = get_dataloader(valid_enc_sci, torch.tensor(validation['label'].to_list()))

In [21]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader_sci:
    input_ids, type_ids, attn_mask, target = batch
    print(input_ids.shape, type_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - SciBERT

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [23]:
class SciBERTclassifier(nn.Module):
    def __init__(self, transformer):
        super(SciBERTclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 2)
    
    def forward(self, ip_ids, type_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask, 
                              token_type_ids=type_ids)
        return  self.linear_layer(op["pooler_output"])

In [24]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [25]:
transformer = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
model = SciBERTclassifier(transformer).to(device)
print(f"The model has {count_parameter(model):,} trainable parameters.")

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 109,920,002 trainable parameters.


In [26]:
# for name, param in model.named_parameters():
#     if "pooler" in name or "linear" in name:#or "layer.11" in name or "layer.10" in name or "linear" in name:
#         param.requires_grad = True
#     else:
#         param.requires_grad = False
#   print(name, param.shape, param.requires_grad)

print(f'The model has {count_parameter(model):,} trainable parameters')

The model has 109,920,002 trainable parameters


In [27]:
# Define optimizer and 
criterion = torch.nn.CrossEntropyLoss()
optim = torch.optim.AdamW(model.parameters(), lr = 2e-5)

#### Train SciBERT Model

In [28]:
def train_model(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch

        optim.zero_grad()
        output = model(input_ids, type_ids, attn_mask)
        loss = criterion(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred)

def evaluate(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, type_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, type_ids, attn_mask)
            loss = criterion(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred), pred, target

In [29]:
best_valid_loss = float('inf')
total_train_loss, total_valid_loss = list(), list()

In [30]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model(model, train_dataloader_sci)
    total_train_loss.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate(model, valid_dataloader_sci)
    total_valid_loss.append(valid_loss)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_pred, best_target = pred, target
        torch.save(model.state_dict(), "model_least_loss.pt")
        print("\nBest Model Saved!!\n")
    
    # elif epoch % 3 == 0:
    torch.save(model.state_dict(), "model_checkpoint_sci" + str(epoch+1) + ".pt")
    print("\nCheckpoint Model Saved!\n")
    
    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:01,  1.58s/it]
2it [00:02,  1.40s/it]
3it [00:04,  1.33s/it]
4it [00:05,  1.29s/it]
5it [00:06,  1.27s/it]
6it [00:07,  1.25s/it]
7it [00:08,  1.24s/it]
8it [00:10,  1.23s/it]
9it [00:11,  1.23s/it]
10it [00:12,  1.23s/it]
11it [00:13,  1.22s/it]
12it [00:15,  1.23s/it]
13it [00:16,  1.23s/it]
14it [00:17,  1.22s/it]
15it [00:18,  1.22s/it]
16it [00:19,  1.23s/it]
17it [00:21,  1.22s/it]
18it [00:22,  1.22s/it]
19it [00:23,  1.23s/it]
20it [00:24,  1.23s/it]
21it [00:26,  1.23s/it]
22it [00:27,  1.23s/it]
23it [00:28,  1.23s/it]
24it [00:29,  1.23s/it]
25it [00:31,  1.23s/it]
26it [00:32,  1.23s/it]
27it [00:33,  1.23s/it]
28it [00:34,  1.23s/it]
29it [00:35,  1.23s/it]
30it [00:37,  1.23s/it]
31it [00:38,  1.23s/it]
32it [00:39,  1.23s/it]
33it [00:40,  1.24s/it]
34it [00:42,  1.24s/it]
35it [00:43,  1.24s/it]
36it [00:44,  1.24s/it]
37it [00:45,  1.24s/it]
38it [00:47,  1.24s/it]
39it [00:48,  1.24s/it]
40it [00:49,  1.


Best Model Saved!!



 20%|██        | 1/5 [04:25<17:41, 265.36s/it]


Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.337 | Train F1 Score: 0.427
Valid Total Loss: 0.345 | Valid F1 Score: 0.444
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.39s/it]
2it [00:02,  1.32s/it]
3it [00:03,  1.29s/it]
4it [00:05,  1.27s/it]
5it [00:06,  1.26s/it]
6it [00:07,  1.26s/it]
7it [00:08,  1.25s/it]
8it [00:10,  1.25s/it]
9it [00:11,  1.25s/it]
10it [00:12,  1.25s/it]
11it [00:13,  1.25s/it]
12it [00:15,  1.24s/it]
13it [00:16,  1.24s/it]
14it [00:17,  1.24s/it]
15it [00:18,  1.25s/it]
16it [00:20,  1.25s/it]
17it [00:21,  1.25s/it]
18it [00:22,  1.25s/it]
19it [00:23,  1.25s/it]
20it [00:25,  1.25s/it]
21it [00:26,  1.24s/it]
22it [00:27,  1.25s/it]
23it [00:28,  1.25s/it]
24it [00:30,  1.25s/it]
25it [00:31,  1.25s/it]
26it [00:32,  1.25s/it]
27it [00:33,  1.25s/it]
28it [00:35,  1.25s/it]
29it [00:36,  1.25s/it]
30it [00:37,  1.25s/it]
31it [00:38,  1.25s/it]
32it [00:40,  1.25s/it]
33it [00:41,  1.25s/it]
34it [00:42,  1.25s/it]
35it [00:43,  1.25s/it]
36it [00:45,  1.25s/it]
37it [00:46,  1.25s/it]
38it [00:47,  1.25s/it]
39it [00:48,  1.25s/it]
40it [00:50,  1.25s/it]
41it [00:51,  1.25s/it]
42it 

...


Checkpoint Model Saved!

Epoch: 03
Train Total Loss: 0.149 | Train F1 Score: 0.818
Valid Total Loss: 0.319 | Valid F1 Score: 0.690
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.28s/it]
2it [00:02,  1.27s/it]
3it [00:03,  1.27s/it]
4it [00:05,  1.27s/it]
5it [00:06,  1.27s/it]
6it [00:07,  1.27s/it]
7it [00:08,  1.27s/it]
8it [00:10,  1.27s/it]
9it [00:11,  1.27s/it]
10it [00:12,  1.26s/it]
11it [00:13,  1.26s/it]
12it [00:15,  1.26s/it]
13it [00:16,  1.26s/it]
14it [00:17,  1.26s/it]
15it [00:18,  1.26s/it]
16it [00:20,  1.26s/it]
17it [00:21,  1.26s/it]
18it [00:22,  1.26s/it]
19it [00:23,  1.26s/it]
20it [00:25,  1.26s/it]
21it [00:26,  1.26s/it]
22it [00:27,  1.26s/it]
23it [00:29,  1.26s/it]
24it [00:30,  1.26s/it]
25it [00:31,  1.26s/it]
26it [00:32,  1.26s/it]
27it [00:34,  1.26s/it]
28it [00:35,  1.26s/it]
29it [00:36,  1.26s/it]
30it [00:37,  1.26s/it]
31it [00:39,  1.26s/it]
32it [00:40,  1.26s/it]
33it [00:41,  1.27s/it]
34it [00:42,  1.27s/it]
35it [00:44,  1.26s/it]
36it [00:45,  1.27s/it]
37it [00:46,  1.27s/it]
38it [00:47,  1.26s/it]
39it [00:49,  1.27s/it]
40it [00:50,  1.26s/it]
41it [00:51,  1.26s/it]
42it 


Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.083 | Train F1 Score: 0.909
Valid Total Loss: 0.483 | Valid F1 Score: 0.675
--------------------



0it [00:00, ?it/s]
1it [00:01,  1.28s/it]
2it [00:02,  1.27s/it]
3it [00:03,  1.27s/it]
4it [00:05,  1.27s/it]
5it [00:06,  1.27s/it]
6it [00:07,  1.27s/it]
7it [00:08,  1.27s/it]
8it [00:10,  1.27s/it]
9it [00:11,  1.27s/it]
10it [00:12,  1.27s/it]
11it [00:13,  1.26s/it]
12it [00:15,  1.27s/it]
13it [00:16,  1.27s/it]
14it [00:17,  1.27s/it]
15it [00:19,  1.27s/it]
16it [00:20,  1.27s/it]
17it [00:21,  1.26s/it]
18it [00:22,  1.27s/it]
19it [00:24,  1.26s/it]
20it [00:25,  1.26s/it]
21it [00:26,  1.26s/it]
22it [00:27,  1.26s/it]
23it [00:29,  1.26s/it]
24it [00:30,  1.26s/it]
25it [00:31,  1.26s/it]
26it [00:32,  1.26s/it]
27it [00:34,  1.27s/it]
28it [00:35,  1.26s/it]
29it [00:36,  1.27s/it]
30it [00:37,  1.26s/it]
31it [00:39,  1.26s/it]
32it [00:40,  1.26s/it]
33it [00:41,  1.26s/it]
34it [00:42,  1.26s/it]
35it [00:44,  1.26s/it]
36it [00:45,  1.26s/it]
37it [00:46,  1.26s/it]
38it [00:48,  1.26s/it]
39it [00:49,  1.26s/it]
40it [00:50,  1.26s/it]
41it [00:51,  1.27s/it]
42it 


Checkpoint Model Saved!

Epoch: 05
Train Total Loss: 0.054 | Train F1 Score: 0.951
Valid Total Loss: 0.519 | Valid F1 Score: 0.635
--------------------


In [31]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.94      0.89      0.92       594
           1       0.59      0.75      0.66       122

    accuracy                           0.87       716
   macro avg       0.77      0.82      0.79       716
weighted avg       0.88      0.87      0.87       716



In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [33]:
from glob import glob

In [34]:
for filepath in glob("*.pt"):
    print(filepath)

model_checkpoint_sci3.pt
model_checkpoint_sci4.pt
model_checkpoint_sci1.pt
model_checkpoint_sci5.pt
model_least_loss.pt
model_checkpoint_sci2.pt


In [36]:
from glob import glob
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/
    time.sleep(10)

In [ ]:
# for filepath in glob("*.pt"):
#     !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/
#     time.sleep(10)